In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

In [ ]:
# setting x dimension
xL = 0
xR = 1
dx = 0.1
x = np.arange(xL, xR + dx, dx)
nx = int((xR - xL) / dx) + 1
x

In [ ]:
# setting y dimension
yB = 0
yT = 1
dy = 0.1
y = np.arange(yB, yT + dy, dy)
ny = int((yT - yB) / dy) + 1
y

In [ ]:
# setting time
t_final = 5  # seconds
dt = 0.05
time = np.arange(0, t_final + dt, dt)
nt = int(t_final / dt)
time

In [ ]:
# physical constants
gamma = 0.1
rhocp = 1.2
tolerance = 1E-4

Dx = gamma * dy / dx
Dy = gamma * dx / dy

Fx = rhocp * dy
Fy = rhocp * dx

In [ ]:
# known flow field
ux = np.zeros((ny, nx))
uy = np.zeros((ny, nx))

for i in range(0, nx):
    for j in range(0, ny):
        ux[j, i] = i * dx
        uy[j, i] = -j * dy

print(ux)
print(uy)

In [ ]:
X, Y = np.meshgrid(x, y, indexing='xy')

plt.title('Velocity Vector')
plt.xlabel('x-position')
plt.ylabel('y-position')
plt.quiver(X, Y, ux, uy, scale=50)
plt.show()

In [ ]:
T = np.zeros((ny, nx, nt + 1))
T_guess = np.zeros((ny, nx))

In [ ]:
# fixed boundary conditions
T[ny-1, :, :] = 2
for j in range(0, ny):
    T[j, 0, :] = (j * dy) + 1

T[:, :, 0]

In [ ]:
plt.title('Initial Temperature Contour')
plt.xlabel('x-position')
plt.ylabel('y-position')
contour = plt.contourf(X, Y, T[:, :, 0])
plt.colorbar()
plt.show()

In [ ]:
n_iterations = 0
error_list = []

for k in range(0, nt):  # time loop
    error = 1
    T_guess[:, :] = T[:, :, k]
    while error > tolerance:
        for j in range(1, ny - 1):  # y loop
            for i in range(1, nx - 1):  # x loop
                T[j, i, k + 1] = (((((Dx + np.amax((-Fx * ux[j, i + 1], 0))) * T[j, i + 1, k + 1]) + 
                                    ((Dx + np.amax((Fx * ux[j, i - 1]))) * T[j, i - 1, k + 1]) +
                                  ((Dy + np.amax((-Fy * uy[j + 1, i], 0))) * T[j + 1, i, k + 1])) + 
                                  ((Dy + np.amax((Fy * uy[j - 1, i], 0))) * T[j - 1, i, k + 1]) + (((rhocp * dx * dx) / dt) * T[j, i, k])) /
                                  ((Dx + np.amax((-Fx * ux[j, i + 1], 0))) + (Dx + np.amax((Fx * ux[j, i - 1]))) + (Dy + np.amax((-Fy * uy[j + 1, i], 0))) + 
                                   (Dy + np.amax((Fy * uy[j - 1, i], 0))) + ((rhocp * dx * dx) / dt)))

        # fixed flux boundary conditions
        # zero flux boundary at the bottom (xy) of the system, there are can be no j - 1 terms, i + 1 and i - 1 stay, j + 1 replaced with 1
        for l in range(1, nx -1):
            T[0, l, k + 1] = (((((Dx + np.amax((-Fx * ux[0, l + 1], 0))) * T[0, l + 1, k + 1]) + ((Dx + np.amax((Fx * ux[0, l - 1]))) * T[0, l - 1, k + 1]) +
                                  ((Dy + np.amax((-Fy * uy[1, l], 0))) * T[1, l, k + 1])) + (((rhocp * dx * dx) / dt) * T[0, l, k])) /
                                  ((Dx + np.amax((-Fx * ux[0, l + 1], 0))) + (Dx + np.amax((Fx * ux[0, l - 1]))) + (Dy + np.amax((-Fy * uy[1, l], 0))) + ((rhocp * dx * dx) / dt)))

        # zero flux boundary condition on right (xy) of the system, there can be no (i + 1) terms, (j - 1) and (j + 1) stay, (i - 1) replaced with (nx - 2) j replaced with m
        for m in range(1, ny -1):
            T[m, nx - 1, k + 1] = (((((Dx + np.amax((Fx * ux[m, nx - 2]))) * T[m, nx - 2, k + 1]) +
                                  ((Dy + np.amax((-Fy * uy[m + 1, nx - 1], 0))) * T[m + 1, nx - 1, k + 1])) + 
                                  ((Dy + np.amax((Fy * uy[m - 1, nx - 1], 0))) * T[m - 1, nx - 1, k + 1]) + (((rhocp * dx * dx) / dt) * T[m, nx - 1, k])) /
                                  ((Dx + np.amax((Fx * ux[m, nx - 2]))) + (Dy + np.amax((-Fy * uy[m + 1, nx - 1], 0))) + 
                                   (Dy + np.amax((Fy * uy[m - 1, nx - 1], 0))) + ((rhocp * dx * dx) / dt)))
            
        #corner BC
        T[0, nx - 1, k + 1] = (((((Dx + np.amax((Fx * ux[0, nx - 2]))) * T[0, nx - 2, k + 1]) +
                                  ((Dy + np.amax((-Fy * uy[1, nx - 1], 0))) * T[1, nx - 1, k + 1])) + (((rhocp * dx * dx) / dt) * T[0, nx - 1, k])) /
                                  ((Dx + np.amax((Fx * ux[0, nx - 2]))) + (Dy + np.amax((-Fy * uy[1, nx - 1], 0))) + ((rhocp * dx * dx) / dt)))



        error = np.amax(np.amax(np.abs((T[:, :, k + 1] - T_guess))))
        error_list.append(error)
        n_iterations += 1
        T_guess[:, :] = T[:, :, k + 1]

In [ ]:
steady_error = []
for k in range(0, nt):
    steady = np.amax(np.amax(np.abs(T[:, :, k + 1] - T[:, :, k])))
    steady_error.append(steady)

In [ ]:
plt.title('Finding Steady State')
plt.xlabel('Time')
plt.ylabel('Error')
plt.yscale('log')
plt.plot(time[1:nt+1], steady_error)
plt.show()

In [ ]:
plt.title('Error vs Iterations')
plt.xlabel('Number of Iterations')
plt.ylabel('Error')
plt.yscale('log')
plt.plot(range(0, n_iterations), error_list)
plt.show()

In [ ]:
print_time = 2
print_time = int(print_time / dt)
print(print_time)

plt.title('Temperature Contour')
plt.xlabel('x-position')
plt.ylabel('y-position')
contour = plt.contourf(X, Y, T[:, :, print_time])
plt.colorbar()
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes()

def animate(i):
    contour = plt.contourf(X, Y, T[:, :, i])
    return contour

anim = animation.FuncAnimation(fig, animate, frames=nt, interval=500, blit=False)

anim.save('test_animation_1.mp4', fps=10)

plt.show()